In [2]:
import json
import pandas as pd
from tqdm import tqdm, trange
import numpy as np
import re
from pprint import pprint
import random
import torch
import gc

In [3]:
with open(f'train.json', 'rb') as f:
    train = json.load(f)

with open(f'test.json', 'rb') as f:
    test = json.load(f)

In [4]:
pprint(train[0])
pprint(test[0])

{'extracted_part': {'answer_end': [1343],
                    'answer_start': [1279],
                    'text': ['Размер обеспечения исполнения контракта 6593.25 '
                             'Российский рубль']},
 'id': 809436509,
 'label': 'обеспечение исполнения контракта',
 'text': 'Извещение о проведении открытого конкурса в электронной форме для '
         'закупки №0328300032822000806 Общая информация Номер извещения '
         '0328300032822000806 Наименование объекта закупки Поставка продуктов '
         'питания Способ определения поставщика (подрядчика, исполнителя) '
         'Открытый конкурс в бль Порядок внесения денежных средств в качестве '
         'обеспечения заявки на участие в закупке, а также условия гарантии '
         'Обеспечение заявки на участие в закупке может предоставляться '
         'участником закупки в виде денежных средств или независимой гарантии, '
         'предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа '
         'обеспечени

In [5]:
for d in train:
  if len(d['extracted_part']['text']) > 1:
    pprint(d)

No multiple extrated parts presentented in train data

# Preprocessing: Punctuation restoration

## Simplest with regex, only full stops

In [ ]:
re.sub(r"([а-я\d]) ([А-Я])", r'\1. \2', train[0]['text'])

'Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806. Общая информация. Номер извещения 0328300032822000806. Наименование объекта закупки. Поставка продуктов питания. Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль. Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии. Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45. Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44. Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45. Федерального закона № 44-ФЗ. Реквизиты счета в соответствии с

In [ ]:
def restore_full_stops(text):
  return re.sub(r"([а-я]) ([А-Я])", r'\1. \2', text)

## With model (not completed), all punctuation

### ru_punct

In [ ]:
!git clone https://github.com/gleb-skobinsky/ru_punct.git
!gdown --output /content/ru_punct/ https://drive.google.com/u/0/uc?id=1jHxW4D7O1V4-h-adJxW27QttYvlpC_w6&export=download

In [ ]:
%cd /content/ru_punct

import nltk
nltk.download('punkt')

!python playing_with_model.py

%cd ../..

/content/ru_punct


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2023-04-16 16:20:22.833368: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 16:20:23.809112: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Vocabulary "content/data_rpocessed/vocabulary" size: 100002
Loading model parameters...
2023-04-16 16:20:25.419488: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-16 16:20:25.448540: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read f

### Siblero

In [ ]:
import torch

model, example_texts, languages, punct, apply_te = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                  model='silero_te')

input_text = train[0]['text']
apply_te(input_text, lan='en')

Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master


&


IndexError: ignored

### FRED-T5

In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.1 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration 


tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-large')
model = T5ForConditionalGeneration.from_pretrained('ai-forever/FRED-T5-large')
device='cuda'
model.to(device)

In [ ]:
text = '<SC1>'
for i, t in enumerate(train[0]['text'].split()):
  text += f'{t}<extra_id_{i}>'
text

'<SC1>Извещение<extra_id_0>о<extra_id_1>проведении<extra_id_2>открытого<extra_id_3>конкурса<extra_id_4>в<extra_id_5>электронной<extra_id_6>форме<extra_id_7>для<extra_id_8>закупки<extra_id_9>№0328300032822000806<extra_id_10>Общая<extra_id_11>информация<extra_id_12>Номер<extra_id_13>извещения<extra_id_14>0328300032822000806<extra_id_15>Наименование<extra_id_16>объекта<extra_id_17>закупки<extra_id_18>Поставка<extra_id_19>продуктов<extra_id_20>питания<extra_id_21>Способ<extra_id_22>определения<extra_id_23>поставщика<extra_id_24>(подрядчика,<extra_id_25>исполнителя)<extra_id_26>Открытый<extra_id_27>конкурс<extra_id_28>в<extra_id_29>бль<extra_id_30>Порядок<extra_id_31>внесения<extra_id_32>денежных<extra_id_33>средств<extra_id_34>в<extra_id_35>качестве<extra_id_36>обеспечения<extra_id_37>заявки<extra_id_38>на<extra_id_39>участие<extra_id_40>в<extra_id_41>закупке,<extra_id_42>а<extra_id_43>также<extra_id_44>условия<extra_id_45>гарантии<extra_id_46>Обеспечение<extra_id_47>заявки<extra_id_48>на<

In [ ]:
lm_text='<SC5>Принялся Кутузов рассказывать свою историю, как он попал в армию<extra_id_0> Началось с того, что он был в армии, служил в артиллерии.'
lm_text = text
input_ids = torch.tensor([tokenizer.encode(lm_text)]).to(device)
outputs = model.generate(input_ids, eos_token_id=tokenizer.eos_token_id, early_stopping=True)

for o in outputs:
  print(tokenizer.decode(o))

<pad> <extra_id_0>  о проведении открытого конкурса ( <extra_id_1> ферта)
 <extra_id_2>  открытого конкурса ( <extra_id_3>  аукциона, <extra_id_4>


# Fine-tuning

## FRED-T5

In [6]:
train_data = []
for d in train:
  ext_d = d['extracted_part']
  X = (d['text'], d['label'])
  y = (ext_d['text'][0], ext_d['answer_start'][0], ext_d['answer_end'][0])
  train_data.append((X, y))

train_data[0]

(('Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета в соответствии с п.16 ч.

In [7]:
pairs = train_data.copy()

In [8]:
!pip install -q transformers sentencepiece bitsandbytes accelerate

In [39]:
from transformers import GPT2Tokenizer, T5Tokenizer, T5ForConditionalGeneration

raw_model = 'cointegrated/rut5-base-multitask'
tokenizer = T5Tokenizer.from_pretrained(raw_model)
model = T5ForConditionalGeneration.from_pretrained(raw_model, device_map='auto', load_in_8bit=True)


# raw_model = 'ai-forever/FRED-T5-large' 
# tokenizer = GPT2Tokenizer.from_pretrained(raw_model)
# model = T5ForConditionalGeneration.from_pretrained(raw_model)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-7)

# device='cuda'
# model.to(device)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


In [40]:
batch_size = 2  # сколько примеров показывем модели за один шаг
report_steps = 100  # раз в сколько шагов печатаем результат
epochs = 3  # сколько раз мы покажем данные модели

In [41]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
model.train()
losses = []
for epoch in range(epochs):
    print('EPOCH', epoch)
    random.shuffle(pairs)
    for i in trange(0, int(len(pairs) / batch_size)):
        batch = pairs[i * batch_size: (i + 1) * batch_size]
        x = tokenizer([f'{p[0][0]} | {p[0][1]}' for p in batch], return_tensors='pt', padding=True).to(model.device)
        y = tokenizer([p[1][0] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y.input_ids[y.input_ids == 0] = -100
        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        if i % report_steps == 0:
            print('step', i, 'loss', np.mean(losses[-report_steps:]))

EPOCH 0


  0%|          | 1/899 [00:01<19:54,  1.33s/it]

step 0 loss 1.1611328125


 11%|█         | 101/899 [00:52<07:28,  1.78it/s]

step 100 loss nan


...

 33%|███▎      | 301/899 [02:34<04:33,  2.19it/s]

step 300 loss nan


 45%|████▍     | 401/899 [03:24<04:04,  2.04it/s]

step 400 loss nan


 53%|█████▎    | 476/899 [04:02<03:52,  1.82it/s]

In [31]:
test_data = []
for d in test:
  X = (d['text'], d['label'])
  test_data.append(f'{X[0]} | {X[1]}')
test_data[0]

'МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание услуг по техническому обслуживанию и ремонту принтеров и многофункциональных устройств, заправке и восстановлению картриджей (идентификационный код закупки 223861800296886010100100590019511244) г. Ханты-Мансийск «___» ____________ 2022 г.  (или) возмещения убытков причинённых Исполнителем убытков. 6. Обеспечение исполнения контракта 6.1. Исполнение контракта обеспечиваются предоставлением независимой гарантии, выданной банком и соответствующей требованиям Федерального закона от 05.04.2013 № 44-ФЗ «О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд», или внесением денежных средств на указанный заказчиком счет, на котором в соответствии с законодательством Российской Федерации учитываются операции со средствами, поступающими заказчику. Способ обеспечения исполнения контракта, срок действия независимой гарантии определяются в соответствии с требованиями Федерального закона от 05.04.2

In [32]:
model.eval()

def answer(x, **kwargs):
    inputs = tokenizer(x, return_tensors='pt').to(model.device) 
    with torch.no_grad():
        hypotheses = model.generate(**inputs, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

In [34]:
sample = train_data[:5]
for x, y in sample:
    print('answer:', y[0])
    print('model: ', answer(f"{x[0]} | {x[1]}"))
    print('---')

answer: Размер обеспечения исполнения контракта 6593.25 Российский рубль


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


model:  
---
answer: Поставщик должен предоставить обеспечение исполнения контракта в размере 10 % от цены Контракта.
model:  
---
answer: Размер обеспечения исполнения контракта 10.00%
model:  
---
answer: Размер обеспечения исполнения контракта 10.00%
model:  
---
answer: Размер обеспечения исполнения контракта 10.00%
model:  
---


## Spacy

In [ ]:
!python -m spacy init fill-config /content/base_config.cfg /content/config.cfg

2023-04-19 14:25:12.459642: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
from sklearn.model_selection import train_test_split

t_data, v_data = train_test_split(data, test_size=0.2, random_state=7)

In [ ]:
import spacy
from spacy.tokens import DocBin


nlp = spacy.blank("ru")
# the DocBin will store the example documents
db = DocBin()
for i, (text, annotations) in enumerate(t_data.to_numpy()):
    doc = nlp(text)
    ents = []
    for label, start, end in annotations:
        try:
          span = doc.char_span(int(start), int(end), label=label)
        except:
          continue
        if span:
          ents.append(span)
    doc.spans['sc'] = ents
    db.add(doc)
db.to_disk("./train.spacy")

In [ ]:
import spacy
from spacy.tokens import DocBin


nlp = spacy.blank("ru")
# the DocBin will store the example documents
db = DocBin()
for i, (text, annotations) in enumerate(v_data.to_numpy()):
    doc = nlp(text)
    ents = []
    for label, start, end in annotations:
        try:
          span = doc.char_span(int(start), int(end), label=label)
        except:
          continue
        if span:
          ents.append(span)
    doc.spans['sc'] = ents
    db.add(doc)
db.to_disk("./dev.spacy")

In [ ]:
!python -m spacy train config.cfg --paths.train /content/train.spacy --paths.dev /content/dev.spacy

2023-04-19 14:27:35.389677: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ No output directory provided
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-04-19 14:27:37,868] [INFO] Set up nlp object from config
[2023-04-19 14:27:37,880] [INFO] Pipeline: ['tok2vec', 'spancat']
[2023-04-19 14:27:37,884] [INFO] Created vocabulary
[2023-04-19 14:27:37,885] [INFO] Finished initializing nlp object
[2023-04-19 14:27:49,144] [INFO] Initialized pipeline components: ['tok2vec', 'spancat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'spancat']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS SPANCAT  SPANS_SC_F  SPANS_SC_P  SPANS_SC_R  SCORE 
---  ------  ------------  ------------  ----------  ----------  ----------  ------
  0       0        187.45       7969.76        0.59        0.30       34.18    0.01
 

# Sentence Embeddings

In [ ]:
!pip install -q razdel navec hnswlib
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
--2023-04-16 15:42:48--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  9.67MB/s    in 6.8s    

2023-04-16 15:42:56 (7.46 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [ ]:
from razdel import sentenize, tokenize
from navec import Navec
import gc
import hnswlib
from collections import Counter

In [ ]:
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

def embed_sentence(sentence):
  e = np.array([navec[word.text] for word in tokenize(sentence) if word.text in navec]).mean(axis=0)
  return e if e.shape == (300,) else None

In [ ]:
def get_kNN_embeddings(embedding: list, index: hnswlib.Index, k: int = 1) -> list:
    labels, distances = index.knn_query(embedding, k=k)
    return labels.T.squeeze()
    #, index.get_items(labels)

In [ ]:
LABELS_EMB = {
    'обеспечение исполнения контракта': embed_sentence('размер обеспечение исполнения контракта'),
    'обеспечение гарантийных обязательств': embed_sentence('обеспечение гарантийных обязательств')
}

In [ ]:
TUNED_LABELS_EMB = {}
most_common_shit = '_%)(5.,:30100,55.0010.00'
for label in LABELS_EMB:
  extracted_part_vocab = Counter()

  for doc in train:
    if doc['label'] == label:
      ext = doc['extracted_part']['text'][0]
      extracted_part_vocab.update([token.text for token in list(tokenize(ext))])
  TUNED_LABELS_EMB[label] = embed_sentence(' '.join([v[0] for v in extracted_part_vocab.most_common(20)]))

In [ ]:
for doc in train[0:10]:
  # Retore full stops
  text = restore_full_stops(doc['text'])
  # Get sentences
  sentences = []
  for sentence in sentenize(text):
    sentences.append(sentence.text)
  N = len(sentences)
  # Get embeddings
  embeddings = np.zeros((N, 300))
  for i, sentence in enumerate(tqdm(sentences, total=N)):
      emb = embed_sentence(sentence)
      if emb is not None:
          embeddings[i, :] = emb
  # Build index
  embedding_index = hnswlib.Index(space='cosine', dim=300)
  embedding_index.init_index(max_elements=N, ef_construction=250, M=16, allow_replace_deleted=True)
  embedding_index.add_items(embeddings, replace_deleted=True)
  # Get closest
  idx = get_kNN_embeddings(TUNED_LABELS_EMB[doc['label']], embedding_index)
  # Result
  print()
  print('kNN closest: ', sentences[idx])
  print('Actual: ', doc['extracted_part']['text'][0])
  print()


  0%|          | 0/22 [00:00<?, ?it/s]<ipython-input-18-5d952dce6cde>:5: RuntimeWarning: Mean of empty slice.
  e = np.array([navec[word.text] for word in tokenize(sentence) if word.text in navec]).mean(axis=0)
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 22/22 [00:00<00:00, 1882.66it/s]



kNN closest:  Способ обеспечения исполнения контракта, гарантийных обязательств, срок действия независимой гарантии определяются в соответствии с требованиями.
Actual:  Размер обеспечения исполнения контракта 6593.25 Российский рубль



100%|██████████| 22/22 [00:00<00:00, 1877.03it/s]



kNN closest:  Поставщик должен предоставить обеспечение исполнения контракта в размере 10 % от цены.
Actual:  Поставщик должен предоставить обеспечение исполнения контракта в размере 10 % от цены Контракта.



100%|██████████| 37/37 [00:00<00:00, 3280.06it/s]


...


kNN closest:  Контракта установлено в размере 212,18, эквивалентном 0,50 % от начальной (максимальной) цены контракта.[footnoteRef:16].
Actual:  Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта



100%|██████████| 23/23 [00:00<00:00, 1520.32it/s]



kNN closest:  Контракта установлено в размере 87,61, эквивалентном 0,50 % от начальной (максимальной) цены контракта.[footnoteRef:16].
Actual:  Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта



100%|██████████| 23/23 [00:00<00:00, 1931.16it/s]


kNN closest:  Контракта установлено в размере 153,30, эквивалентном 0,50 % от начальной (максимальной) цены контракта.[footnoteRef:16].
Actual:  Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта



# QA

In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 21.7 MB/s eta 0:00:00


In [ ]:
import torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering


tokenizer = AutoTokenizer.from_pretrained("timpal0l/mdeberta-v3-base-squad2")
model = AutoModelForQuestionAnswering.from_pretrained("timpal0l/mdeberta-v3-base-squad2")

In [ ]:
from transformers import GPT2Tokenizer, T5ForConditionalGeneration



tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-large',eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained('ai-forever/FRED-T5-large')

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("AndrewChar/model-QA-5-epoch-RU")
model = AutoModelForQuestionAnswering.from_pretrained("AndrewChar/model-QA-5-epoch-RU", from_tf=True)

All TF 2.0 model weights were used when initializing DistilBertForQuestionAnswering.

All the weights of DistilBertForQuestionAnswering were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForQuestionAnswering for predictions without further training.


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")

model = AutoModelForQuestionAnswering.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")

In [ ]:
sample = train[1110]
sample

{'id': 947961169,
 'text': 'Извещение о проведении электронного аукциона для закупки №0133300020022000149 Общая информация Номер извещения 0133300020022000149 Наименование объекта закупки Компьютеры и периферийное оборудование (покупка компьютерного оборудования) Способ определения поставщика (подрядчика, исполнителя) Электронный аукцион но. При этом срок действия независимой гарантии должен превышать предусмотренный контрактом срок исполнения обязательств, которые должны быть обеспечены такой независимой гарантией, не менее чем на один месяц, в том числе в случае его изменения в соответствии со статьей 95 Федерального закона № 44-ФЗ. Платежные реквизиты "Номер расчётного счёта" 40102810645370000025 "Номер лицевого счёта" 05333021560 "БИК" 012406500 "Наименование кредитной организации" ОТДЕЛЕНИЕ ИВАНОВСКОГО БАНКА РОССИИ//УФК ПО ИВАНОВСКОЙ ОБЛАСТИ "Номер корреспондентского счета" 03232643246070003300 Требования к гарантии качества товара, работы, услуги Требуется гарантия качества товар

In [ ]:
question, text = f'Пункт про {sample["label"]}', sample['text']
question

'Пункт про обеспечение гарантийных обязательств'

In [ ]:
inputs = tokenizer(question, text, return_tensors="pt", max_length=512, padding='max_length', truncation=True)
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
print(answer_start_index, answer_end_index)

tensor(310) tensor(312)


In [ ]:
tokenizer.decode(predict_answer_tokens)

'Требуется'